In [1]:
import torch
from torch.nn import functional as F
import numpy as np
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
from rsna19.configs.second_level import Config
from sklearn.metrics import log_loss
import pandas as pd
from scipy.signal import windows

In [2]:
# train_folds = [0, 1, 2, 3] # for testing 
train_folds = [0, 1, 2, 3, 4]  # for stage 2 submission
val_folds = [4]

cache_dir = '/home/dmytro/ml/kaggle-rsna-2019/output/cache'

train_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in train_folds], dim=0)
train_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in train_folds], dim=0)
val_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in val_folds], dim=0)
val_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in val_folds], dim=0)

n_models = 5
class_weights = torch.tensor([1, 1, 1, 1, 1, 2], dtype=torch.float32) * 6 / 7
loss_fn = F.binary_cross_entropy

In [3]:
# undo sigmoid

# train_x[train_x > 0] = torch.log(train_x[train_x > 0] / (1-train_x[train_x > 0]))
# val_x[val_x > 0] = torch.log(val_x[val_x > 0] / (1-val_x[val_x > 0]))
# loss_fn = F.binary_cross_entropy_with_logits

In [4]:
print('train')
preds = []
for model_id in range(n_models):
    preds.append(train_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], train_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, train_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

print('\nval')
preds = []
for model_id in range(n_models):
    preds.append(val_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], val_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, val_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

train
model 0: 0.06113557517528534
model 1: 0.062465518712997437
model 2: 0.0627981498837471
model 3: 0.061474189162254333
model 4: 0.06128436699509621
averaged ensemble: 0.05732988193631172

val
model 0: 0.061941273510456085
model 1: 0.06328877061605453
model 2: 0.06281528621912003
model 3: 0.06388995796442032
model 4: 0.06146755814552307
averaged ensemble: 0.05845646187663078


In [5]:
# train on middle slice only

# train_x_1slice = []
# for model_id in range(n_models):
#     train_x_1slice.append(train_x[:, model_id*30+12:model_id*30+18])
# train_x_1slice = torch.cat(train_x_1slice, dim=1)

# val_x_1slice = []
# for model_id in range(n_models):
#     val_x_1slice.append(val_x[:, model_id*30+12:model_id*30+18])
# val_x_1slice = torch.cat(val_x_1slice, dim=1)

# train_x = train_x_1slice
# val_x = val_x_1slice

In [11]:
hidden = 128
features_out = 6

# model = torch.nn.Sequential(
#         torch.nn.Linear(train_x.shape[1], features_out)
# )

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = torch.nn.Linear(train_x.shape[1], features_out, bias=False)

    def forward(self, x):
        x = F.linear(x, torch.abs(self.w1.weight) / torch.sum(torch.abs(self.w1.weight), 1, keepdim=True), self.w1.bias)
#         x = F.linear(x, self.w1.weight / torch.sum(self.w1.weight, 1, keepdim=True), self.w1.bias)
        return torch.clamp(x, 0, 1)


model = Model()
print(model.w1.weight.shape)
# model = torch.nn.Sequential(
#         torch.nn.Linear(train_x.shape[1], hidden),
#         torch.nn.ReLU(),
# #         torch.nn.Dropout(0.2),
#         torch.nn.Linear(hidden, features_out),
# )

train_x = train_x.cuda()
train_y = train_y.cuda()
val_x = val_x.cuda()
val_y = val_y.cuda()
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), 0.0001)
val_log_loss = 0
class_weights = class_weights.cuda()

for i in tqdm(range(10000)):
    optimizer.zero_grad()

    y_hat = model(train_x)
    loss = F.binary_cross_entropy(y_hat, train_y, weight=class_weights)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()
        val_y_hat = model(val_x)
        val_loss = F.binary_cross_entropy(val_y_hat, val_y, weight=class_weights)
        model.train()
        
        print(f'{i:04d}: train: {loss.item():.04f}, val: {val_loss.item():.04f}')


  0%|          | 22/10000 [00:00<00:45, 218.01it/s]

torch.Size([6, 150])
0000: train: 0.1309, val: 0.1316



  2%|▏         | 155/10000 [00:00<00:38, 256.62it/s]

0100: train: 0.1205, val: 0.1211



  2%|▏         | 238/10000 [00:00<00:36, 266.88it/s]

...


 98%|█████████▊| 9843/10000 [00:36<00:00, 257.85it/s]

9800: train: 0.0573, val: 0.0582



 99%|█████████▉| 9947/10000 [00:37<00:00, 257.21it/s]

9900: train: 0.0573, val: 0.0582



100%|██████████| 10000/10000 [00:37<00:00, 268.34it/s]

In [9]:
torch.abs(model.w1.weight) / torch.sum(torch.abs(model.w1.weight), 1, keepdim=True)

tensor([[1.6175e-05, 3.3427e-06, 7.2556e-06, 3.6481e-06, 3.5664e-06, 5.8942e-06,
         1.0334e-02, 7.9139e-06, 2.6752e-06, 4.0633e-06, 6.9975e-06, 5.5327e-06,
         7.7679e-03, 1.2487e-05, 3.5652e-06, 1.8475e-06, 5.6836e-06, 7.4572e-06,
         6.7314e-06, 2.8986e-07, 6.8215e-06, 2.6493e-06, 6.9987e-06, 3.5359e-06,
         1.1278e-05, 7.9612e-06, 5.5704e-06, 6.9655e-07, 9.4248e-06, 8.0391e-06,
         7.0075e-02, 5.2306e-06, 7.8575e-06, 3.7447e-06, 3.6100e-06, 2.4426e-06,
         8.0505e-02, 9.6311e-07, 9.8790e-06, 5.6635e-06, 1.0270e-05, 2.7731e-06,
         1.1803e-01, 1.0128e-05, 3.4024e-06, 7.1866e-06, 6.4559e-06, 5.4386e-06,
         3.5133e-02, 9.3689e-06, 9.1701e-06, 4.9436e-06, 4.0378e-06, 5.7272e-06,
         1.2139e-05, 6.4111e-06, 3.2622e-07, 7.0553e-06, 7.3319e-06, 3.8639e-06,
         2.4847e-02, 2.6180e-06, 3.7442e-06, 3.7545e-06, 3.4401e-06, 7.0550e-06,
         4.1661e-02, 7.4850e-06, 1.6732e-06, 4.0225e-06, 7.1409e-06, 2.6647e-06,
         4.1517e-02, 5.8188e

Apply the trained model on the L1 test predictions:

TODO: implement apply to test and call generate_submission with clip_eps 1e-5